In [1]:
import requests
import re
from bs4 import BeautifulSoup

In [12]:
hltb = requests.get('https://howlongtobeat.com/game.php?id=4250').content
hltb_soup = BeautifulSoup(hltb, "lxml")
capture_title = hltb_soup.find('div', attrs = {'class': 'profile_header shadow_text'})
game_title = capture_title.get_text().strip('\n').strip(' $')
game_title

'Half-Life 2: Episode One'

In [8]:
## Need to generate numbers to find all titles, including 4 and 5 digits

In [18]:
hltb_soup.find_all('tr')

[<tr>
 <td>Single-Player</td>
 <td class="center" style="width: 50px;">Polled</td> <td>Average</td>
 <td>Median</td>
 <td>Rushed</td>
 <td>Leisure</td>
 </tr>, <tr>
 <td>Main Story</td>
 <td class="center time_100_text">723 </td> <td>3h 57m </td>
 <td>4h </td>
 <td>2h 41m </td>
 <td>6h 59m </td>
 </tr>, <tr>
 <td>Main + Extras</td>
 <td class="center time_100_text">198 </td> <td>4h 27m </td>
 <td>4h </td>
 <td>3h 06m </td>
 <td>7h 44m </td>
 </tr>, <tr>
 <td>Completionists</td>
 <td class="center time_100_text">172 </td> <td>5h 34m </td>
 <td>5h </td>
 <td>3h 48m </td>
 <td>18h 38m </td>
 </tr>, <tr>
 <td>All PlayStyles</td>
 <td class="center time_100_text">1.1K </td> <td>4h 17m </td>
 <td>4h </td>
 <td>2h 46m </td>
 <td>18h 04m </td>
 </tr>, <tr>
 <td>Speedrun</td>
 <td class="center" style="width: 50px;">Polled</td> <td>Average</td>
 <td>Median</td>
 <td>Fastest</td>
 <td>Slowest</td>
 </tr>, <tr>
 <td>Any%</td>
 <td class="center time_50_text">5 </td> <td>2h 05m 16s</td>
 <td>2h 15